In [1]:
import numpy as np
import pandas as pd


data_folder  = '../data/titanic/preprocessed/'

In [2]:
X_train = pd.read_csv(data_folder + 'X_train.csv', index_col=False)
X_test = pd.read_csv(data_folder + 'X_test.csv', index_col=False)

In [3]:
import pickle

with open(data_folder + 'y_train.pkl', 'rb') as handle:
    y_train = pickle.load(handle)
    
with open(data_folder + 'y_test.pkl', 'rb') as handle:
    y_test = pickle.load(handle)

In [4]:
y_train.shape

(623,)

In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [79]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 8)
        self.fc2 = nn.Linear(8, 5)
        self.fc3 = nn.Linear(5, 8)
        self.fc4 = nn.Linear(8, 10)
        self.relu = F.relu
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        
        hidden_output = x
        
        x = self.fc4(x)
        
        output = x
        
        return hidden_output, output

In [80]:
input_dim = 10


model = Autoencoder(input_dim)

In [81]:
print(len(list(model.parameters())))
print(list(model.parameters())[0].size())
print(list(model.parameters())[1].size())
print(list(model.parameters())[2].size())
print(list(model.parameters())[3].size())

8
torch.Size([8, 10])
torch.Size([8])
torch.Size([5, 8])
torch.Size([5])


In [82]:
criterion = nn.MSELoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [83]:
y_train = y_train.values.reshape(-1, 1)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [84]:
X_train.shape, y_train.shape

((623, 10), (623, 1))

In [85]:
num_epochs = 100000

In [86]:
for epoch in range(num_epochs):
    x_torch = torch.from_numpy(X_train.values).type(torch.FloatTensor)
    y_torch = torch.from_numpy(X_train.values).type(torch.FloatTensor)
#     y_torch = torch.from_numpy(y_train).type(torch.FloatTensor)
    x = Variable(x_torch)
    y = Variable(y_torch)

    optimizer.zero_grad()

    _, outputs = model(x)
    cost = criterion(outputs, y)
    cost.backward()

    optimizer.step()
    
    if epoch % 10000 == 0:
        print(((y_torch - outputs.data)**2).mean())

0.6957991585450826
0.02730451139838433
0.02730442510047522
0.027304434210262556
0.027304424661406364
0.027304435133846002
0.027304740695326724
0.027304630649337795
0.02730445047101667
0.027308247283282728


In [87]:
X_train_hidden, _ = model(Variable(torch.from_numpy(X_train.values).type(torch.FloatTensor)))
X_test_hidden, _ = model(Variable(torch.from_numpy(X_test.values).type(torch.FloatTensor)))

In [88]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)

/home/calvinku/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [89]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_test, y_pred[:,1]))

0.8317708333333333


In [90]:
X_train_hidden = X_train_hidden.data.numpy()
X_test_hidden = X_test_hidden.data.numpy()


lr = LogisticRegression()

lr.fit(X_train_hidden, y_train)
y_pred = lr.predict_proba(X_test_hidden)

/home/calvinku/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [91]:
print(roc_auc_score(y_test, y_pred[:,1]))

0.8052083333333334
